In [16]:
import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from PIL import Image
import glob
import os
import math
import csv
import os


In [17]:
lat_lon_file_path='/Users/moctader/Thesis_code/lat_lon/results.csv'
feature_vector_path='/Users/moctader/Thesis_code/lat_lon/feature_vector.csv'
csv_file_path = '/Users/moctader/Thesis_code/lat_lon/results.csv'
tiles_template = "/Users/moctader/Thesis/{t}/{z}/{x}/{y}.png"
combined_csv_path = '/Users/moctader/Thesis_code/lat_lon/combined_results.csv'
points = gpd.read_file("/Users/moctader/Thesis_code/GTK_ASsoil_obs.csv")
t_values = glob.glob('/Users/moctader/Thesis/*')
base_directory = "/Users/moctader/Thesis_code/output20/"


In [18]:
points.POINT_X = points.POINT_X.astype("float")
points.POINT_Y = points.POINT_Y.astype("float")
zoom_level = 10



In [19]:
samples = gpd.GeoDataFrame(
    points.CLASS, crs="EPSG:3067", geometry=gpd.points_from_xy(points.POINT_X, points.POINT_Y)
).to_crs("WGS84")

tile_list = [(point.x, point.y) for point in samples['geometry']]


In [20]:
samples["i"] = samples.index
samples["filenames"] = samples.apply(lambda row: f"{row['CLASS']}/image_{row['i']}", axis=1)
samples



,CLASS,geometry,i,filenames
0,ASS,POINT (25.76894 64.77799),0,ASS/image_0
1,ASS,POINT (25.77630 64.79350),1,ASS/image_1
2,ASS,POINT (25.78469 64.78681),2,ASS/image_2
3,ASS,POINT (25.31512 64.98873),3,ASS/image_3
4,ASS,POINT (25.34464 64.79449),4,ASS/image_4
...,...,...,...,...
5819,ASS,POINT (25.50228 65.64123),5819,ASS/image_5819
5820,non-ASS,POINT (25.37067 65.62436),5820,non-ASS/image_5820
5821,non-ASS,POINT (25.37108 65.62473),5821,non-ASS/image_5821
5822,ASS,POINT (25.34070 65.58217),5822,ASS/image_5822


In [21]:
def get_lat_from_row(p):
    lon, lat = p.geometry.x, p.geometry.y
    return lat

def get_lon_from_one_column(geometry):
    # geometry is a column
    lon, lat = geometry.x, geometry.y
    return lon

In [22]:
samples["lat"] = samples.apply(lambda row: get_lat_from_row(row), axis=1)

In [23]:
samples["lon"] = samples["geometry"].map(get_lon_from_one_column)


In [24]:
samples

,CLASS,geometry,i,filenames,lat,lon
0,ASS,POINT (25.76894 64.77799),0,ASS/image_0,64.777988,25.768938
1,ASS,POINT (25.77630 64.79350),1,ASS/image_1,64.793496,25.776304
2,ASS,POINT (25.78469 64.78681),2,ASS/image_2,64.786808,25.784691
3,ASS,POINT (25.31512 64.98873),3,ASS/image_3,64.988732,25.315115
4,ASS,POINT (25.34464 64.79449),4,ASS/image_4,64.794488,25.344641
...,...,...,...,...,...,...
5819,ASS,POINT (25.50228 65.64123),5819,ASS/image_5819,65.641230,25.502278
5820,non-ASS,POINT (25.37067 65.62436),5820,non-ASS/image_5820,65.624358,25.370668
5821,non-ASS,POINT (25.37108 65.62473),5821,non-ASS/image_5821,65.624730,25.371078
5822,ASS,POINT (25.34070 65.58217),5822,ASS/image_5822,65.582168,25.340697


In [25]:
def load_data(filename, directory):
    path = directory + "/" + filename + ".png"
    return np.array(Image.open(path))[:, :, :3]  # return 3 channels
    
    

In [26]:
# Get a list of file_names (files and directories) in the base directory
file_names = os.listdir(base_directory)

# Filter out non-directory file_names
files = [file for file in file_names if os.path.isdir(os.path.join(base_directory, file))]

for single_file in files:
    samples[single_file] = samples["filenames"].map(
        lambda name, directory=os.path.join(base_directory, single_file): load_data(name, directory)
    )


In [27]:
samples

,CLASS,geometry,i,filenames,lat,lon,elev_10m_hillshade,landsat,elev_10m_slope,aem_real_component,corine_land_cover,elev_10m,aem_apparent_resistivity,aem_imaginary_component,TPI,elev_10m_aspect
0,ASS,POINT (25.76894 64.77799),0,ASS/image_0,64.777988,25.768938,"[[[197, 197, 197], [196, 196, 196], [197, 197,...","[[[234, 121, 245], [163, 30, 166], [72, 10, 10...","[[[255, 255, 255], [254, 254, 254], [254, 254,...","[[[231, 24, 25], [231, 26, 26], [231, 24, 25],...","[[[3, 3, 3], [3, 3, 3], [25, 25, 25], [2, 2, 2...","[[[246, 246, 246], [246, 246, 246], [246, 246,...","[[[253, 52, 29], [254, 56, 28], [254, 56, 27],...","[[[232, 48, 48], [228, 12, 15], [228, 10, 11],...","[[[129, 129, 129], [127, 127, 127], [128, 128,...","[[[125, 125, 125], [120, 120, 120], [147, 147,..."
1,ASS,POINT (25.77630 64.79350),1,ASS/image_1,64.793496,25.776304,"[[[196, 196, 196], [197, 197, 197], [197, 197,...","[[[113, 180, 121], [123, 190, 167], [133, 209,...","[[[255, 255, 255], [255, 255, 255], [255, 255,...","[[[230, 25, 25], [230, 23, 22], [230, 21, 20],...","[[[34, 34, 34], [17, 17, 17], [17, 17, 17], [1...","[[[253, 253, 253], [253, 253, 253], [253, 253,...","[[[255, 61, 26], [254, 55, 26], [254, 53, 26],...","[[[228, 4, 5], [229, 5, 5], [229, 5, 5], [229,...","[[[128, 128, 128], [130, 130, 130], [129, 129,...","[[[85, 85, 85], [143, 143, 143], [111, 111, 11..."
2,ASS,POINT (25.78469 64.78681),2,ASS/image_2,64.786808,25.784691,"[[[196, 196, 196], [196, 196, 196], [196, 196,...","[[[134, 138, 143], [183, 133, 146], [172, 134,...","[[[253, 253, 253], [253, 253, 253], [255, 255,...","[[[229, 24, 24], [230, 25, 25], [232, 27, 27],...","[[[5, 5, 5], [17, 17, 17], [17, 17, 17], [2, 2...","[[[252, 252, 252], [253, 253, 253], [253, 253,...","[[[254, 62, 28], [255, 60, 28], [255, 61, 27],...","[[[228, 5, 5], [228, 5, 5], [228, 5, 5], [227,...","[[[131, 131, 131], [128, 128, 128], [130, 130,...","[[[81, 81, 81], [68, 68, 68], [71, 71, 71], [9..."
3,ASS,POINT (25.31512 64.98873),3,ASS/image_3,64.988732,25.315115,"[[[196, 196, 196], [196, 196, 196], [196, 196,...","[[[1, 4, 45], [1, 4, 45], [1, 4, 45], [1, 4, 4...","[[[255, 255, 255], [255, 255, 255], [255, 255,...","[[[254, 253, 254], [254, 253, 254], [254, 253,...","[[[49, 49, 49], [49, 49, 49], [49, 49, 49], [4...","[[[255, 255, 255], [255, 255, 255], [255, 255,...","[[[255, 254, 253], [255, 254, 253], [255, 254,...","[[[255, 254, 255], [255, 254, 255], [255, 254,...","[[[128, 128, 128], [128, 128, 128], [128, 128,...","[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
4,ASS,POINT (25.34464 64.79449),4,ASS/image_4,64.794488,25.344641,"[[[196, 196, 196], [196, 196, 196], [196, 196,...","[[[3, 2, 86], [5, 1, 94], [3, 4, 74], [2, 4, 7...","[[[255, 255, 255], [255, 255, 255], [255, 255,...","[[[255, 254, 255], [255, 254, 255], [255, 254,...","[[[49, 49, 49], [49, 49, 49], [49, 49, 49], [4...","[[[255, 255, 255], [255, 255, 255], [255, 255,...","[[[253, 254, 254], [253, 254, 254], [253, 254,...","[[[253, 254, 254], [253, 254, 254], [253, 254,...","[[[128, 128, 128], [128, 128, 128], [128, 128,...","[[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5819,ASS,POINT (25.50228 65.64123),5819,ASS/image_5819,65.641230,25.502278,"[[[196, 196, 196], [195, 195, 195], [196, 196,...","[[[100, 103, 107], [97, 122, 110], [109, 196, ...","[[[255, 255, 255], [254, 254, 254], [255, 255,...","[[[138, 138, 245], [138, 141, 241], [138, 139,...","[[[25, 25, 25], [25, 25, 25], [25, 25, 25], [4...","[[[185, 185, 185], [186, 186, 186], [186, 186,...","[[[175, 254, 84], [165, 254, 93], [136, 255, 1...","[[[14, 14, 229], [16, 16, 230], [18, 18, 230],...","[[[128, 128, 128], [130, 130, 130], [129, 129,...","[[[107, 107, 107], [65, 65, 65], [77, 77, 77],..."
5820,non-ASS,POINT (25.37067 65.62436),5820,non-ASS/image_5820,65.624358,25.370668,"[[[198, 198, 198], [198, 198, 198], [198, 198,...","[[[183, 172, 118], [245, 210, 175], [245, 216,...","[[[255, 255, 255], [255, 2

In [30]:
import numpy as np
import cv2
import pandas as pd


# Initialize an empty list to store combined images for all rows
combined_images_list = []

# Iterate over rows of the DataFrame
for index, row in samples.iterrows():
    # Initialize an empty list to store individual layers
    layers_list = []

    # Iterate over the selected columns
    for single_file in files:
        layer = (row[single_file])
        layers_list.append(layer)

    # Concatenate the layers along the third dimension to create a multi-channel image
    combined_image = np.concatenate(layers_list, axis=-1)
    
    # Append the combined image to the list
    combined_images_list.append(combined_image)

# Add the combined images as a new column in the DataFrame

samples["combined_channels"] = combined_images_list

# Print the updated DataFrame
print(samples.head(2))


  CLASS                   geometry  i    filenames        lat        lon  \
0   ASS  POINT (25.76894 64.77799)  0  ASS/image_0  64.777988  25.768938   
1   ASS  POINT (25.77630 64.79350)  1  ASS/image_1  64.793496  25.776304   

                                  elev_10m_hillshade  \
0  [[[197, 197, 197], [196, 196, 196], [197, 197,...   
1  [[[196, 196, 196], [197, 197, 197], [197, 197,...   

                                             landsat  \
0  [[[234, 121, 245], [163, 30, 166], [72, 10, 10...   
1  [[[113, 180, 121], [123, 190, 167], [133, 209,...   

                                      elev_10m_slope  \
0  [[[255, 255, 255], [254, 254, 254], [254, 254,...   
1  [[[255, 255, 255], [255, 255, 255], [255, 255,...   

                                  aem_real_component  \
0  [[[231, 24, 25], [231, 26, 26], [231, 24, 25],...   
1  [[[230, 25, 25], [230, 23, 22], [230, 21, 20],...   

                                   corine_land_cover  \
0  [[[3, 3, 3], [3, 3, 3], [25, 25, 25]

In [29]:
# Check the images chanels

base_directory = "/Users/moctader/Thesis_code/output20/"

# Get a list of file_names in the base directory
file_names = os.listdir(base_directory)

# Filter out non-directory file_names
files = [file for file in file_names if os.path.isdir(os.path.join(base_directory, file))]

for single_file in files:
    #mage_file_path=f'/Users/moctader/Thesis_code/output20/{single_file}/non-ASS/image_5786.png'
    image_file_path=f'/Users/moctader/Thesis_code/output20/{single_file}/ASS/image_0.png'
    image = Image.open(image_file_path)

    # Get the bands (channels) of the image
    bands = image.getbands()
    
    # Determine the number of channels
    num_channels = len(bands)

    # Print the result
    print(f"The image at {image_file_path} has {num_channels} channel(s): {bands}")

The image at /Users/moctader/Thesis_code/output20/elev_10m_hillshade/ASS/image_0.png has 3 channel(s): ('R', 'G', 'B')
The image at /Users/moctader/Thesis_code/output20/landsat/ASS/image_0.png has 3 channel(s): ('R', 'G', 'B')
The image at /Users/moctader/Thesis_code/output20/elev_10m_slope/ASS/image_0.png has 3 channel(s): ('R', 'G', 'B')
The image at /Users/moctader/Thesis_code/output20/aem_real_component/ASS/image_0.png has 3 channel(s): ('R', 'G', 'B')
The image at /Users/moctader/Thesis_code/output20/corine_land_cover/ASS/image_0.png has 3 channel(s): ('R', 'G', 'B')
The image at /Users/moctader/Thesis_code/output20/elev_10m/ASS/image_0.png has 3 channel(s): ('R', 'G', 'B')
The image at /Users/moctader/Thesis_code/output20/aem_apparent_resistivity/ASS/image_0.png has 3 channel(s): ('R', 'G', 'B')
The image at /Users/moctader/Thesis_code/output20/aem_imaginary_component/ASS/image_0.png has 3 channel(s): ('R', 'G', 'B')
The image at /Users/moctader/Thesis_code/output20/TPI/ASS/image